In [3]:
from sympy import symbols
from typing import Dict

class RecurrenceSubstitutionSolver:
    """
    Pure substitution method - unroll recurrence relations until reaching base case
    """
    
    def __init__(self):
        self.n = symbols('n', positive=True, integer=True)
        self.k = symbols('k', positive=True, integer=True)
        self.i = symbols('i', integer=True)
        self.steps = []
        
    def clear_steps(self):
        """Clear the step history"""
        self.steps = []
    
    def add_step(self, level: int, description: str, expression: str, explanation: str = ""):
        """Add a substitution step"""
        step = {
            'level': level,
            'description': description,
            'expression': expression,
            'explanation': explanation
        }
        self.steps.append(step)
        
    def print_step(self, step: Dict):
        """Print a single step with formatting"""
        indent = "  " * step['level']
        print(f"\n{indent}Level {step['level']}: {step['description']}")
        print(f"{indent}  {step['expression']}")
        if step['explanation']:
            print(f"{indent}  → {step['explanation']}")
    
    def print_all_steps(self):
        """Print all recorded steps"""
        print("\n" + "="*70)
        print("SUBSTITUTION METHOD - UNROLLING TO BASE CASE")
        print("="*70)
        
        for step in self.steps:
            self.print_step(step)
        
        print("\n" + "="*70)
    
    def substitute_recurrence(self, a: int, b: int, f_expr, base_case_value=1, max_levels=None):
        """
        Substitute recurrence T(n) = a*T(n/b) + f(n) until reaching T(1)
        
        Args:
            a: coefficient of recursive term
            b: divisor in recursive call  
            f_expr: the non-recursive part f(n)
            base_case_value: value of T(1)
            max_levels: maximum substitution levels (auto-detect if None)
        """
        self.clear_steps()
        
        # Determine number of levels needed to reach base case
        if max_levels is None:
            # We need n/b^k = 1, so k = log_b(n)
            max_levels = 5  # Show first 5 levels explicitly
        
        # Step 0: Original recurrence
        original = f"T(n) = {a}*T(n/{b}) + {self._format_function(f_expr)}"
        self.add_step(0, "Original Recurrence", original, 
                     "Starting with the given recurrence relation")
        
        # Step 1: First substitution
        level1_substitution = f"T(n/{b}) = {a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}"
        level1_result = f"T(n) = {a}*[{a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}] + {self._format_function(f_expr)}"
        level1_expanded = f"T(n) = {a**2}*T(n/{b**2}) + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(1, "First Substitution", level1_substitution,
                     f"Substitute T(n/{b}) using the recurrence relation")
        self.add_step(1, "Substitute Back", level1_result,
                     "Replace T(n/b) in the original equation")  
        self.add_step(1, "Expand", level1_expanded,
                     "Distribute and simplify")
        
        # Step 2: Second substitution
        level2_substitution = f"T(n/{b**2}) = {a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}"
        level2_result = f"T(n) = {a**2}*[{a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}] + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        level2_expanded = f"T(n) = {a**3}*T(n/{b**3}) + {a**2}*{self._format_function(f_expr, f'n/{b**2}')} + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(2, "Second Substitution", level2_substitution,
                     f"Continue substituting T(n/{b**2})")
        self.add_step(2, "Substitute Back", level2_result,
                     "Replace in the expanded expression")
        self.add_step(2, "Expand", level2_expanded,
                     "Distribute and collect terms")
        
        # Step 3: Pattern recognition
        pattern = f"T(n) = {a}^k * T(n/{b}^k) + Σ(i=0 to k-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        self.add_step(3, "Pattern Recognition", pattern,
                     "After k substitutions, we get this general form")
        
        # Step 4: Determine when to stop (base case)
        base_condition = f"n/{b}^k = 1"
        solve_k = f"k = log_{b}(n)"
        self.add_step(4, "Base Case Condition", base_condition,
                     "We stop when the argument reaches 1")
        self.add_step(4, "Solve for k", solve_k,
                     "Number of substitutions needed")
        
        # Step 5: Substitute base case
        base_substitution = f"T(n) = {a}^(log_{b}(n)) * T(1) + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        with_base_value = f"T(n) = {a}^(log_{b}(n)) * {base_case_value} + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(5, "Apply Base Case", base_substitution,
                     "Replace T(1) with its value")
        self.add_step(5, "With Base Value", with_base_value,
                     f"T(1) = {base_case_value}")
        
        # Step 6: Simplify the first term
        if a == b:
            first_term_simplified = f"{a}^(log_{b}(n)) = n"
        else:
            first_term_simplified = f"{a}^(log_{b}(n)) = n^(log_{b}({a}))"
        
        self.add_step(6, "Simplify First Term", first_term_simplified,
                     "Use the property a^(log_b(n)) = n^(log_b(a))")
        
        # Step 7: Evaluate the sum based on f(n)
        self._evaluate_sum(f_expr, a, b, base_case_value)
        
        return self.steps
    
    def _format_function(self, f_expr, arg='n'):
        """Format the function expression with given argument"""
        if f_expr == 1:
            return "1"
        elif f_expr == self.n:
            return arg
        elif f_expr == self.n**2:
            return f"({arg})²"
        elif f_expr == self.n**3:
            return f"({arg})³"
        elif str(f_expr) == "n*log(n)":
            return f"{arg}*log({arg})"
        else:
            # Replace n with the argument
            return str(f_expr).replace('n', arg)
    
    def _evaluate_sum(self, f_expr, a, b, base_case_value):
        """Evaluate the sum part based on the type of f(n)"""
        
        if f_expr == 1:  # f(n) = 1 (constant)
            self._evaluate_constant_sum(a, b, base_case_value)
        elif f_expr == self.n:  # f(n) = n (linear)
            self._evaluate_linear_sum(a, b, base_case_value)
        elif f_expr == self.n**2:  # f(n) = n² (quadratic)
            self._evaluate_quadratic_sum(a, b, base_case_value)
        else:
            self._evaluate_general_sum(f_expr, a, b, base_case_value)
    
    def _evaluate_constant_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = 1"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i"
        self.add_step(7, "Sum with f(n) = 1", sum_expr,
                     "The sum becomes a geometric series")
        
        if a == 1:
            result = f"log_{b}(n)"
            complexity = "O(log n)"
        else:
            geometric_formula = f"({a}^(log_{b}(n)) - 1) / ({a} - 1)"
            if a == b:
                result = f"(n - 1) / ({a} - 1)"
                complexity = "O(n)"
            else:
                result = f"(n^(log_{b}({a})) - 1) / ({a} - 1)"
                if a > b:
                    complexity = f"O(n^(log_{b}({a})))"
                else:
                    complexity = "O(1)"
        
        self.add_step(7, "Geometric Series Result", result,
                     "Using geometric series formula")
        
        # Final result
        if a == b:
            if base_case_value == 1:
                final = f"T(n) = n + (n-1)/({a}-1) = O(n)"
            else:
                final = f"T(n) = {base_case_value}*n + (n-1)/({a}-1) = O(n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_linear_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)"
        factored = f"n * Σ(i=0 to log_{b}(n)-1) ({a}/{b})^i"
        
        self.add_step(7, "Sum with f(n) = n", sum_expr,
                     "Extract n and evaluate the ratio sum")
        self.add_step(7, "Factor out n", factored,
                     "Factor out n from the sum")
        
        ratio = a / b
        if ratio == 1:  # a = b
            result = f"n * log_{b}(n)"
            complexity = "O(n log n)"
        elif ratio > 1:  # a > b
            geometric_result = f"n * ({ratio}^(log_{b}(n)) - 1) / ({ratio} - 1)"
            # Since ratio^(log_b(n)) = (a/b)^(log_b(n)) = n^(log_b(a/b)) = n^(log_b(a) - 1)
            simplified = f"n * (n^(log_{b}({a}) - 1) - 1) / ({ratio} - 1)"
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b
            result = f"n * (1 - ({ratio})^(log_{b}(n))) / (1 - {ratio})"
            complexity = "O(n)"
        
        self.add_step(7, "Evaluate Geometric Series", result,
                     f"Ratio r = {a}/{b} = {ratio}")
        
        # Final result combining both terms
        if a == b:
            if base_case_value == 1:
                final = f"T(n) = n + n*log_{b}(n) = O(n log n)"
            else:
                final = f"T(n) = {base_case_value}*n + n*log_{b}(n) = O(n log n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_quadratic_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n²"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)²"
        factored = f"n² * Σ(i=0 to log_{b}(n)-1) ({a}/{b}²)^i"
        
        self.add_step(7, "Sum with f(n) = n²", sum_expr,
                     "Extract n² and evaluate the ratio sum")
        self.add_step(7, "Factor out n²", factored,
                     "Factor out n² from the sum")
        
        ratio = a / (b**2)
        if abs(ratio - 1) < 1e-10:  # a = b²
            result = f"n² * log_{b}(n)"
            complexity = "O(n² log n)"
        elif ratio > 1:  # a > b²
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b²
            complexity = "O(n²)"
        
        self.add_step(7, "Evaluate Sum", f"Ratio = {a}/{b}² = {ratio}",
                     "Determine the dominant term")
        
        final = f"T(n) = {complexity}"
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_general_sum(self, f_expr, a, b, base_case_value):
        """Handle general f(n) expressions"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(7, f"Sum with f(n) = {f_expr}", sum_expr,
                     "Evaluate this sum based on the specific function")
        
        final = "T(n) = [Depends on specific analysis of the sum]"
        self.add_step(8, "Final Result", final, 
                     "Complete analysis requires evaluating the specific sum")

def demonstrate_substitution_examples():
    """
    Demonstrate pure substitution method on classic examples
    """
    solver = RecurrenceSubstitutionSolver()
    
    print("PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE")
    print("=" * 60)
    
    # Example 1: Binary Search - T(n) = T(n/2) + 1
    print("\nEXAMPLE 1: BINARY SEARCH")
    print("T(n) = T(n/2) + 1, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=1, b=2, f_expr=1, base_case_value=1)
    solver.print_all_steps()
    
    # Example 2: Merge Sort - T(n) = 2T(n/2) + n  
    print("\n\nEXAMPLE 2: MERGE SORT")
    print("T(n) = 2T(n/2) + n, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=2, b=2, f_expr=solver.n, base_case_value=1)
    solver.print_all_steps()


if __name__ == "__main__":
    demonstrate_substitution_examples()

PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE

EXAMPLE 1: BINARY SEARCH
T(n) = T(n/2) + 1, T(1) = 1
----------------------------------------

SUBSTITUTION METHOD - UNROLLING TO BASE CASE

Level 0: Original Recurrence
  T(n) = 1*T(n/2) + 1
  → Starting with the given recurrence relation

  Level 1: First Substitution
    T(n/2) = 1*T(n/4) + 1
    → Substitute T(n/2) using the recurrence relation

  Level 1: Substitute Back
    T(n) = 1*[1*T(n/4) + 1] + 1
    → Replace T(n/b) in the original equation

  Level 1: Expand
    T(n) = 1*T(n/4) + 1*1 + 1
    → Distribute and simplify

    Level 2: Second Substitution
      T(n/4) = 1*T(n/8) + 1
      → Continue substituting T(n/4)

    Level 2: Substitute Back
      T(n) = 1*[1*T(n/8) + 1] + 1*1 + 1
      → Replace in the expanded expression

    Level 2: Expand
      T(n) = 1*T(n/8) + 1*1 + 1*1 + 1
      → Distribute and collect terms

      Level 3: Pattern Recognition
        T(n) = 1^k * T(n/2^k) + Σ(i=0 to k-1) 1^i * 1
        → After k 

In [6]:
import sympy as sp
from sympy import symbols, log, simplify, expand, factor, ceiling, floor, Sum, Pow
from typing import Dict, List, Tuple
import math

class RecurrenceSubstitutionSolver:
    """
    Pure substitution method - unroll recurrence relations until reaching base case
    """
    
    def __init__(self):
        self.n = symbols('n', positive=True, integer=True)
        self.k = symbols('k', positive=True, integer=True)
        self.i = symbols('i', integer=True)
        self.steps = []
        
    def clear_steps(self):
        """Clear the step history"""
        self.steps = []
    
    def add_step(self, level: int, description: str, expression: str, explanation: str = ""):
        """Add a substitution step"""
        step = {
            'level': level,
            'description': description,
            'expression': expression,
            'explanation': explanation
        }
        self.steps.append(step)
        
    def print_step(self, step: Dict):
        """Print a single step with formatting"""
        indent = "  " * step['level']
        print(f"\n{indent}Level {step['level']}: {step['description']}")
        print(f"{indent}  {step['expression']}")
        if step['explanation']:
            print(f"{indent}  → {step['explanation']}")
    
    def print_all_steps(self):
        """Print all recorded steps"""
        print("\n" + "="*70)
        print("SUBSTITUTION METHOD - UNROLLING TO BASE CASE")
        print("="*70)
        
        for step in self.steps:
            self.print_step(step)
        
        print("\n" + "="*70)
    
    def substitute_recurrence(self, a: int, b: int, f_expr, base_case_value=1, max_levels=None):
        """
        Substitute recurrence T(n) = a*T(n/b) + f(n) until reaching T(1)
        
        Args:
            a: coefficient of recursive term
            b: divisor in recursive call  
            f_expr: the non-recursive part f(n)
            base_case_value: value of T(1)
            max_levels: maximum substitution levels (auto-detect if None)
        """
        self.clear_steps()
        
        # Determine number of levels needed to reach base case
        if max_levels is None:
            # We need n/b^k = 1, so k = log_b(n)
            max_levels = 5  # Show first 5 levels explicitly
        
        # Step 0: Original recurrence
        original = f"T(n) = {a}*T(n/{b}) + {self._format_function(f_expr)}"
        self.add_step(0, "Original Recurrence", original, 
                     "Starting with the given recurrence relation")
        
        # Step 1: First substitution
        level1_substitution = f"T(n/{b}) = {a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}"
        level1_result = f"T(n) = {a}*[{a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}] + {self._format_function(f_expr)}"
        level1_expanded = f"T(n) = {a**2}*T(n/{b**2}) + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(1, "First Substitution", level1_substitution,
                     f"Substitute T(n/{b}) using the recurrence relation")
        self.add_step(1, "Substitute Back", level1_result,
                     "Replace T(n/b) in the original equation")  
        self.add_step(1, "Expand", level1_expanded,
                     "Distribute and simplify")
        
        # Step 2: Second substitution
        level2_substitution = f"T(n/{b**2}) = {a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}"
        level2_result = f"T(n) = {a**2}*[{a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}] + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        level2_expanded = f"T(n) = {a**3}*T(n/{b**3}) + {a**2}*{self._format_function(f_expr, f'n/{b**2}')} + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(2, "Second Substitution", level2_substitution,
                     f"Continue substituting T(n/{b**2})")
        self.add_step(2, "Substitute Back", level2_result,
                     "Replace in the expanded expression")
        self.add_step(2, "Expand", level2_expanded,
                     "Distribute and collect terms")
        
        # Step 3: Pattern recognition
        pattern = f"T(n) = {a}^k * T(n/{b}^k) + Σ(i=0 to k-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        self.add_step(3, "Pattern Recognition", pattern,
                     "After k substitutions, we get this general form")
        
        # Step 4: Determine when to stop (base case)
        base_condition = f"n/{b}^k = 1"
        solve_k = f"k = log_{b}(n)"
        self.add_step(4, "Base Case Condition", base_condition,
                     f"We stop when the argument reaches 1")
        self.add_step(4, "Solve for k", solve_k,
                     f"Number of substitutions needed")
        
        # Step 5: Substitute base case
        base_substitution = f"T(n) = {a}^(log_{b}(n)) * T(1) + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        with_base_value = f"T(n) = {a}^(log_{b}(n)) * {base_case_value} + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(5, "Apply Base Case", base_substitution,
                     f"Replace T(1) with its value")
        self.add_step(5, "With Base Value", with_base_value,
                     f"T(1) = {base_case_value}")
        
        # Step 6: Simplify the first term
        if a == 1:
            first_term_simplified = f"{a}^(log_{b}(n)) = 1"
            explanation = f"Since 1 raised to any power equals 1"
        elif a == b:
            first_term_simplified = f"{a}^(log_{b}(n)) = n"
            explanation = f"Since {a}^(log_{a}(n)) = n"
        else:
            first_term_simplified = f"{a}^(log_{b}(n)) = n^(log_{b}({a}))"
            explanation = f"Use the property a^(log_b(n)) = n^(log_b(a))"
        
        self.add_step(6, "Simplify First Term", first_term_simplified, explanation)
        
        # Step 7: Evaluate the sum based on f(n)
        self._evaluate_sum(f_expr, a, b, base_case_value)
        
        return self.steps
    
    def _format_function(self, f_expr, arg='n'):
        """Format the function expression with given argument"""
        if f_expr == 1:
            return "1"
        elif f_expr == self.n:
            return arg
        elif f_expr == self.n**2:
            return f"({arg})²"
        elif f_expr == self.n**3:
            return f"({arg})³"
        elif str(f_expr) == "n*log(n)":
            return f"{arg}*log({arg})"
        else:
            # Replace n with the argument
            return str(f_expr).replace('n', arg)
    
    def _evaluate_sum(self, f_expr, a, b, base_case_value):
        """Evaluate the sum part based on the type of f(n)"""
        
        if f_expr == 1:  # f(n) = 1 (constant)
            self._evaluate_constant_sum(a, b, base_case_value)
        elif f_expr == self.n:  # f(n) = n (linear)
            self._evaluate_linear_sum(a, b, base_case_value)
        elif f_expr == self.n**2:  # f(n) = n² (quadratic)
            self._evaluate_quadratic_sum(a, b, base_case_value)
        else:
            self._evaluate_general_sum(f_expr, a, b, base_case_value)
    
    def _evaluate_constant_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = 1"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i"
        self.add_step(7, "Sum with f(n) = 1", sum_expr,
                     "The sum becomes a geometric series")
        
        if a == 1:
            result = f"log_{b}(n)"
            complexity = "O(log n)"
            geometric_explanation = f"Since {a}^i = 1 for all i, we sum 1 exactly log_{b}(n) times"
        else:
            geometric_formula = f"({a}^(log_{b}(n)) - 1) / ({a} - 1)"
            if a == b:
                result = f"(n - 1) / ({a} - 1)"
                complexity = "O(n)"
                geometric_explanation = f"Geometric series with ratio {a}, sum = {geometric_formula}"
            else:
                result = f"(n^(log_{b}({a})) - 1) / ({a} - 1)"
                if a > b:
                    complexity = f"O(n^(log_{b}({a})))"
                else:
                    complexity = "O(1)"
                geometric_explanation = f"Geometric series with ratio {a}, sum = {geometric_formula}"
        
        self.add_step(7, "Geometric Series Result", result, geometric_explanation)
        
        # Final result
        if a == 1:
            if base_case_value == 1:
                final = f"T(n) = 1 + log_{b}(n) = O(log n)"
            else:
                final = f"T(n) = {base_case_value} + log_{b}(n) = O(log n)"
        elif a == b:
            if base_case_value == 1:
                final = f"T(n) = n + (n-1)/({a}-1) = O(n)"
            else:
                final = f"T(n) = {base_case_value}*n + (n-1)/({a}-1) = O(n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_linear_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)"
        factored = f"n * Σ(i=0 to log_{b}(n)-1) ({a}/{b})^i"
        
        self.add_step(7, "Sum with f(n) = n", sum_expr,
                     "Extract n and evaluate the ratio sum")
        self.add_step(7, "Factor out n", factored,
                     f"Factor out n from the sum")
        
        ratio = a / b
        if ratio == 1:  # a = b
            result = f"n * log_{b}(n)"
            complexity = "O(n log n)"
        elif ratio > 1:  # a > b
            geometric_result = f"n * ({ratio}^(log_{b}(n)) - 1) / ({ratio} - 1)"
            # Since ratio^(log_b(n)) = (a/b)^(log_b(n)) = n^(log_b(a/b)) = n^(log_b(a) - 1)
            simplified = f"n * (n^(log_{b}({a}) - 1) - 1) / ({ratio} - 1)"
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b
            result = f"n * (1 - ({ratio})^(log_{b}(n))) / (1 - {ratio})"
            complexity = "O(n)"
        
        self.add_step(7, "Evaluate Geometric Series", result,
                     f"Ratio r = {a}/{b} = {ratio}")
        
        # Final result combining both terms
        if a == b:
            if base_case_value == 1:
                final = f"T(n) = n + n*log_{b}(n) = O(n log n)"
            else:
                final = f"T(n) = {base_case_value}*n + n*log_{b}(n) = O(n log n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_quadratic_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n²"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)²"
        factored = f"n² * Σ(i=0 to log_{b}(n)-1) ({a}/{b}²)^i"
        
        self.add_step(7, "Sum with f(n) = n²", sum_expr,
                     "Extract n² and evaluate the ratio sum")
        self.add_step(7, "Factor out n²", factored,
                     f"Factor out n² from the sum")
        
        ratio = a / (b**2)
        if abs(ratio - 1) < 1e-10:  # a = b²
            result = f"n² * log_{b}(n)"
            complexity = "O(n² log n)"
        elif ratio > 1:  # a > b²
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b²
            complexity = "O(n²)"
        
        self.add_step(7, "Evaluate Sum", f"Ratio = {a}/{b}² = {ratio}",
                     f"Determine the dominant term")
        
        final = f"T(n) = {complexity}"
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_general_sum(self, f_expr, a, b, base_case_value):
        """Handle general f(n) expressions"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(7, f"Sum with f(n) = {f_expr}", sum_expr,
                     "Evaluate this sum based on the specific function")
        
        final = "T(n) = [Depends on specific analysis of the sum]"
        self.add_step(8, "Final Result", final, 
                     "Complete analysis requires evaluating the specific sum")

def demonstrate_substitution_examples():
    """
    Demonstrate pure substitution method on classic examples
    """
    solver = RecurrenceSubstitutionSolver()
    
    print("PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE")
    print("=" * 60)
    
    # Example 1: Binary Search - T(n) = T(n/2) + 1
    print("\nEXAMPLE 1: BINARY SEARCH")
    print("T(n) = T(n/2) + 1, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=1, b=2, f_expr=1, base_case_value=1)
    solver.print_all_steps()
    
    # Example 2: Merge Sort - T(n) = 2T(n/2) + n  
    print("\n\nEXAMPLE 2: MERGE SORT")
    print("T(n) = 2T(n/2) + n, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=2, b=2, f_expr=solver.n, base_case_value=1)
    solver.print_all_steps()
    

if __name__ == "__main__":
    demonstrate_substitution_examples()

PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE

EXAMPLE 1: BINARY SEARCH
T(n) = T(n/2) + 1, T(1) = 1
----------------------------------------

SUBSTITUTION METHOD - UNROLLING TO BASE CASE

Level 0: Original Recurrence
  T(n) = 1*T(n/2) + 1
  → Starting with the given recurrence relation

  Level 1: First Substitution
    T(n/2) = 1*T(n/4) + 1
    → Substitute T(n/2) using the recurrence relation

  Level 1: Substitute Back
    T(n) = 1*[1*T(n/4) + 1] + 1
    → Replace T(n/b) in the original equation

  Level 1: Expand
    T(n) = 1*T(n/4) + 1*1 + 1
    → Distribute and simplify

    Level 2: Second Substitution
      T(n/4) = 1*T(n/8) + 1
      → Continue substituting T(n/4)

    Level 2: Substitute Back
      T(n) = 1*[1*T(n/8) + 1] + 1*1 + 1
      → Replace in the expanded expression

    Level 2: Expand
      T(n) = 1*T(n/8) + 1*1 + 1*1 + 1
      → Distribute and collect terms

      Level 3: Pattern Recognition
        T(n) = 1^k * T(n/2^k) + Σ(i=0 to k-1) 1^i * 1
        → After k 